In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
import gc
import random
import datetime
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

import transformers
from transformers import BertTokenizer, BertForSequenceClassification 
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import AdamW
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

from tensorflow.keras.metrics import AUC, Accuracy
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/Users/muratdemiralay/Downloads/dataset_train.csv")

In [ ]:
df.label.value_counts()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df["label"].replace({-1:2}, inplace=True)

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state = 42)
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
print("Train data size: ", train_df.size)
print("Validation data size: ", val_df.size)

In [ ]:
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
train_df.dropna(inplace=True)
val_df.dropna(inplace=True)

In [ ]:
L_RATE = 1e-5
MAX_LEN = 50
NUM_EPOCHS = 3
BATCH_SIZE = 64 

In [ ]:
#This used for only premise model, rest of the training is same

def custome_encoders(df, labels = None):

    input_ids = []
    attention_masks = []

    for index, row in df.iterrows():
        
        encoded_dict = tokenizer.encode_plus(
                            row["premise"],             
                            add_special_tokens = True,
                            max_length = MAX_LEN,
                            truncation = True,
                            pad_to_max_length = True,
                            return_attention_mask = True, 
                            return_tensors = 'pt',
                       )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    if labels is not None:
        labels = torch.tensor(labels)
        return input_ids, attention_masks, labels
    else:
        return input_ids, attention_masks

In [ ]:
def custome_encoders(df, labels = None):

    input_ids = []
    attention_masks = []

    for index, row in df.iterrows():
        
        encoded_dict = tokenizer.encode_plus(
                            row["premise"],row["hypothesis"],               
                            add_special_tokens = True,
                            max_length = MAX_LEN,
                            truncation = True,
                            pad_to_max_length = True,
                            return_attention_mask = True, 
                            return_tensors = 'pt',
                       )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    if labels is not None:
        labels = torch.tensor(labels)
        return input_ids, attention_masks, labels
    else:
        return input_ids, attention_masks

In [ ]:
batch_size = 64

In [ ]:
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

training_labels = train_df.label.values
validation_labels = val_df.label.values

tr_input_ids,tr_attention_masks,tr_labels = custome_encoders(train_df, labels = training_labels)
val_input_ids,val_attention_masks,val_labels = custome_encoders(val_df, labels = validation_labels)

train_dataset = TensorDataset(tr_input_ids, tr_attention_masks, tr_labels)
validation_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                        batch_size = batch_size,
                                        sampler = RandomSampler(train_dataset))

val_dataloader = torch.utils.data.DataLoader(validation_dataset,
                                            sampler = SequentialSampler(validation_dataset),
                                        batch_size = batch_size)

In [ ]:
number_of_categories = len(train_df['label'].unique())

In [ ]:
import torch.nn as nn
from transformers import XLMRobertaModel

class RobertaModel(nn.Module):
    
    def __init__(self, num_labels):
        super(RobertaModel, self).__init__()
        model_name = "roberta-base"
        self.roberta = XLMRobertaModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.2)
        self.classifier = nn.Sequential(
            nn.Linear(768, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_labels))
        
        self.loss = nn.CrossEntropyLoss()
        self.num_labels = num_labels

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        output = self.dropout(output.pooler_output)
        logits = self.classifier(output)

        if labels is not None:
            loss = self.loss(logits.view(-1, self.num_labels), labels.view(-1))
            return {"loss": loss, "logits": logits}
        else:
            return logits

In [ ]:
model = RobertaModel(num_labels=3) 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

In [ ]:
epochs = 3

optimizer = AdamW(model.parameters(),
                  lr = 1e-5,
                  eps = 1e-8)

total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
def flat_accuracy(preds, labels):
    
    """A function for calculating accuracy scores"""
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    return accuracy_score(labels_flat, pred_flat)

def flat_f1(preds, labels):
    
    """A function for calculating f1 scores"""
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    return f1_score(labels_flat, pred_flat, average = 'macro')

In [ ]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
training_stats = []
total_t0 = time.time()


for epoch_i in range(0, epochs):

        
    print('')
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    
    t0 = time.time()
    
    total_train_loss = 0

    
    model.train()
    
    for step, batch in enumerate(train_dataloader):
        

        if step % 200 == 0 and not step == 0:
            
            elapsed = format_time(time.time() - t0)
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        
        b_input_ids = batch[0].to(device).to(torch.int64)
        b_input_mask = batch[1].to(device).to(torch.int64)
        b_labels = batch[2].to(device).to(torch.int64)
        
        model.zero_grad() 

        
        outputs = model(b_input_ids, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

        loss = outputs["loss"]
        logits = outputs["logits"]
        total_train_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        
        optimizer.step()
        
        scheduler.step()
    
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    training_time = format_time(time.time() - t0)

    print('')
    print('  Average training loss: {0:.2f}'.format(avg_train_loss))
    print('  Training epcoh took: {:}'.format(training_time))

    print('')
    print('Running Validation...')

    t0 = time.time()
    
    model.eval()
    
    total_eval_accuracy = 0
    total_eval_loss = 0
    total_eval_f1 = 0
    nb_eval_steps = 0

    
    for batch in val_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        
        with torch.no_grad():        

            outputs = model(b_input_ids, 
                            attention_mask=b_input_mask,
                            labels=b_labels)
    
        
        loss = outputs["loss"]
        logits = outputs["logits"]
        total_eval_loss += loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        total_eval_f1 += flat_f1(logits, label_ids)
        

    
    avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
    print('  Accuracy: {0:.2f}'.format(avg_val_accuracy))
    
    
    avg_val_f1 = total_eval_f1 / len(val_dataloader)
    print('  F1: {0:.2f}'.format(avg_val_f1))
    
    avg_val_loss = total_eval_loss / len(val_dataloader)
    
    
    validation_time = format_time(time.time() - t0)
    
    print('  Validation Loss: {0:.2f}'.format(avg_val_loss))
    print('  Validation took: {:}'.format(validation_time))

    
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Val_F1' : avg_val_f1,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print('')
print('Training complete!')